<a href="https://colab.research.google.com/github/airinrs/Fake-News-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [ ]:
data_fake = pd.read_csv('Fake.csv')
data_true = pd.read_csv('True.csv')


In [ ]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
data_fake['class'] = 0
data_true['class'] = 1

In [ ]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [ ]:
# Take last 10 fake news samples for manual testing
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480, 23470, -1):
    data_fake.drop([i], axis=0, inplace=True)

# Take last 10 real news samples for manual testing
data_true_manual_testing = data_true.tail(10)
for i in range(21416, 21406, -1):
    data_true.drop([i], axis=0, inplace=True)


In [ ]:
data_fake.shape, data_true.shape

((23471, 5), (21407, 5))

In [ ]:
data_fake_manual_testing['class'] = 0
data_true_manual_testing['class'] = 1

/tmp/ipython-input-277247672.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class'] = 0
/tmp/ipython-input-277247672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class'] = 1


In [ ]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [ ]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


Merging true and fake news

In [ ]:
data_merge = pd.concat([data_fake, data_true], axis = 0) # stacking vertically (adding rows)
data_merge.head(10)
# This creates one big dataset (data_merge) containing all articles, with fake and true mixed together.

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [ ]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [ ]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [ ]:
data.isnull().sum()

,0
text,0
class,0


After merging, we shuffle (data.sample(frac=1)) so the fake and true articles are randomly mixed.

In [ ]:
data = data.sample(frac = 1)

In [ ]:
data.head()

,text,class
14432,LONDON (Reuters) - Britain will submit proposa...,1
765,Donald Trump s son-in-law Jared Kushner has fo...,0
18507,Protesters affiliated with the group No Justic...,0
23272,21st Century Wire says If you have only recent...,0
14518,PRISTINA (Reuters) - A court in Kosovo on Frid...,1


In [ ]:
data.reset_index(inplace = True) # reset the index numbering and star fresh
data.drop(['index'], axis = 1, inplace = True)

#index=True modifies the DataFrame directly instead of returning a copy.
# axis = 1 operates horizontal and removes

In [ ]:
data.columns

Index(['text', 'class'], dtype='object')

In [ ]:
data.head(5)

,text,class
0,LONDON (Reuters) - Britain will submit proposa...,1
1,Donald Trump s son-in-law Jared Kushner has fo...,0
2,Protesters affiliated with the group No Justic...,0
3,21st Century Wire says If you have only recent...,0
4,PRISTINA (Reuters) - A court in Kosovo on Frid...,1


In [ ]:
def wordopt(text):
  text = text.lower() # Converts all text to lowercase.
  text = re.sub('\[.*?\]', '', text)
  text = re.sub("\\W", " ", text)
  text = re.sub('https?://\S+|www\.S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w', '', text)
  return text

In [ ]:
data['text'] = data['text'].apply(wordopt)

In [ ]:
x = data['text']
y = data['class']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

 converting words into numerical features using TF-IDF

TF = Term Frequency (how often a word appears in a document).

IDF = Inverse Document Frequency (how rare that word is across all documents).

Fit → Learn the vocabulary from x_train and compute IDF values for each word.

Transform → Convert each training document into a TF-IDF vector using that vocabulary.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

Note: Why we need xv_* instead of x_*
 original x_train and x_test are raw strings (news articles).

Machine learning models like Logistic Regression or Decision Trees cannot work directly on text — they need numbers.

So we use TF-IDF vectorization (TfidfVectorizer) to convert text into numeric feature vectors.

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, y_test)

0.9857397504456328

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5889
           1       0.98      0.99      0.98      5331

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



Trying decision tree algorithm





In [ ]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred_dt = DT.predict(xv_test)

In [ ]:
DT.score(xv_test, y_test)

0.9951871657754011

In [ ]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5889
           1       1.00      0.99      0.99      5331

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



Gradient boosting is an ensemble learning algorithm that produces accurate predictions by combining multiple decision trees into a single model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
predict_gb = GB.predict(xv_test)

In [ ]:
GB.score(xv_test, y_test)

0.9955436720142602

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rf = RF.predict(xv_test)

In [ ]:
RF.score(xv_test, y_test)

0.9871657754010695

In [ ]:
print(classification_report, y_test, pred_rf)

<function classification_report at 0x7e12f939d080> 10878    1
26675    0
6192     1
15019    0
11983    1
        ..
21002    1
23513    0
12006    0
27533    0
33201    1
Name: class, Length: 11220, dtype: int64 [1 0 1 ... 0 0 1]


In [ ]:
def output_label(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"

def manual_testing(news):
  testing_news = {"text": [news]} #Creates a dictionary where the key is "text" and the value is a list containing the news string.
  new_def_test = pd.DataFrame(testing_news) #Converts the dictionary into a DataFrame so it matches the format used in training.
  new_def_test["text"] = new_def_test["text"].apply(wordopt) # Clean the text
  new_x_test = new_def_test["text"]
  new_xv_test = vectorization.transform(new_x_test)
  pred_LR = LR.predict(new_xv_test)
  pred_DR = DT.predict(new_xv_test)
  pred_GBC = GB.predict(new_xv_test)
  pred_RFC = RF.predict(new_xv_test)

  return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                            output_label(pred_DR[0]),
                                                                                                         output_label(pred_GBC[0]),
                                                                                                         output_label(pred_RFC[0])))

Detect a fake news

In [ ]:
news = str(input())
manual_testing(news)


Republicans are trying to rig the midterms. Will Democrats actually fight back, or cave?: A conversation with Beto O’Rourke


LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


Detect a true news

In [ ]:
news = str(input())
manual_testing(news)



. The United Kingdom ended 2019 with clarity about Brexit, but it took a turbulent journey to get there. The year started with the country facing a March 29 deadline for leaving the European Union (EU). Prime Minister Theresa May chose that date but couldn’t persuade the House of Commons to approve the deal she struck with the EU. The main sticking point was the “backstop” provision, which avoided creating a customs barrier in the middle of the Irish Sea but saddled Britain with EU custom rules. May was forced to delay Brexit until October 31, and then resigned after the House of Commons voted down her deal three times. Boris Johnson won a Conservative Party vote and became prime minister on July 24. He struck a new deal that swapped the backstop for a customs barrier between Northern Ireland and the rest of the United Kingdom. 


LR Prediction: True News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: True News


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nbformat

# Path to your notebook
nb_path = "/content/drive/MyDrive/Colab Notebooks/FakeNewsDetection.ipynb"

# Load notebook
with open(nb_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove broken widget metadata if it exists
if "widgets" in nb["metadata"]:
    del nb["metadata"]["widgets"]

# Save back
with open(nb_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)
